# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import pandas as pd

# Load the CSV file
df = pd.read_csv('homepage_actions.csv')

# Display the first few rows
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
# Check columns, data types, and missing values
df.info()

# Summary stats
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [5]:
# Unique users
unique_users = df['id'].nunique()
print(f'Total unique users: {unique_users}')

# Total rows
print(f'Total rows: {len(df)}')


Total unique users: 6328
Total rows: 8188


In [6]:
# Check distribution of actions
df['action'].value_counts()


view     6328
click    1860
Name: action, dtype: int64

In [7]:
# Get users who clicked
clicked_users = set(df[df['action'] == 'click']['id'])

# Get users who viewed
viewed_users = set(df[df['action'] == 'view']['id'])

# Check for users who clicked without viewing
clicked_without_view = clicked_users - viewed_users
print(f'Users who clicked without viewing: {len(clicked_without_view)}')


Users who clicked without viewing: 0


In [8]:
# Number of users in each group
df['group'].value_counts()

# Check if any user appears in both groups
user_groups = df.groupby('id')['group'].nunique()
overlap_users = user_groups[user_groups > 1]
print(f'Number of users in both groups: {len(overlap_users)}')


Number of users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [9]:
#Your code here

# Calculate clicks and views pre group
# Total views per group
views = df[df['action'] == 'view'].groupby('group')['id'].nunique()
print(f"Total Views:\n{views}\n")

# Total clicks per group
clicks = df[df['action'] == 'click'].groupby('group')['id'].nunique()
print(f"Total Clicks:\n{clicks}\n")

# Conversion rates (clicks/views)
conversion_rates = clicks / views
print(f"Conversion Rates:\n{conversion_rates}\n")

# performing z-test
from statsmodels.stats.proportion import proportions_ztest

# Successes = Number of clicks
successes = [clicks['control'], clicks['experiment']]

# Observations = Number of views
nobs = [views['control'], views['experiment']]

# One-tailed test (testing if experiment > control)
z_stat, p_val = proportions_ztest(count=successes, nobs=nobs, alternative='larger')

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_val:.4f}")



Total Views:
group
control       3332
experiment    2996
Name: id, dtype: int64

Total Clicks:
group
control       932
experiment    928
Name: id, dtype: int64

Conversion Rates:
group
control       0.279712
experiment    0.309746
Name: id, dtype: float64

Z-statistic: -2.6186
P-value: 0.9956


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [11]:
#Your code here
# Get number of views and clicks per group
views = df[df['action'] == 'view'].groupby('group')['id'].nunique()
clicks = df[df['action'] == 'click'].groupby('group')['id'].nunique()

print(f"Views per group:\n{views}\n")
print(f"Clicks per group:\n{clicks}\n")

# Control group's click-through rate
control_ctr = clicks['control'] / views['control']
print(f"Control CTR: {control_ctr:.4f}")

# Expected clicks in experiment group
expected_clicks_exp = control_ctr * views['experiment']
print(f"Expected clicks in Experiment group (if same CTR as control): {expected_clicks_exp:.2f}")


Views per group:
group
control       3332
experiment    2996
Name: id, dtype: int64

Clicks per group:
group
control       932
experiment    928
Name: id, dtype: int64

Control CTR: 0.2797
Expected clicks in Experiment group (if same CTR as control): 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [12]:
#Your code here
import numpy as np

# Control CTR (from Step 1)
control_ctr = clicks['control'] / views['control']

# Observed clicks in experiment group
observed_clicks_exp = clicks['experiment']

# Variance and standard deviation
n_exp = views['experiment']
variance = n_exp * control_ctr * (1 - control_ctr)
std_dev = np.sqrt(variance)

print(f"Standard Deviation: {std_dev:.2f}")

# Z-score
expected_clicks_exp = control_ctr * n_exp
z_score = (observed_clicks_exp - expected_clicks_exp) / std_dev

print(f"Z-score: {z_score:.4f}")


Standard Deviation: 24.57
Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [13]:
#Your code here
from scipy.stats import norm

# Calculate p-value
p_value = 1 - norm.cdf(z_score)

print(f"P-value: {p_value:.4f}")


P-value: 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **NO, The initial z-test gave a non-significant result (p = 0.996), while the manual verification yielded a highly significant result (p = 0.0001)**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.